## Import

In [77]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

import lightning.pytorch as pl
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor


from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [103]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':50,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED': 41,
    'DIR' : 'experiment',
    'EXP_NAME' : 'baseline'
}

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [6]:
all_img_list = glob.glob('train/*/*')
# all_img_list = [f.replace(os.sep, '/') for f in all_img_list]

In [8]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split(os.sep)[-2])

In [9]:
df

,img_path,label
0,train\가구수정\0.png,가구수정
1,train\가구수정\1.png,가구수정
2,train\가구수정\10.png,가구수정
3,train\가구수정\11.png,가구수정
4,train\가구수정\2.png,가구수정
...,...,...
3452,train\훼손\995.png,훼손
3453,train\훼손\996.png,훼손
3454,train\훼손\997.png,훼손
3455,train\훼손\998.png,훼손


In [18]:
train_data, val_data, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [19]:
le = preprocessing.LabelEncoder()

train_data['label'] = le.fit_transform(train_data['label'])

train_data['label_multi_hot'] = np.eye(len(le.classes_), dtype = np.int32)[train_data['label'].tolist()].tolist()

val_data['label'] = le.transform(val_data['label'])
val_data['label_multi_hot'] = np.eye(len(le.classes_), dtype = np.int32)[val_data['label'].tolist()].tolist()

## weighted random sampler

In [ ]:
WeightedRandomSampler()

## CustomDataset

In [22]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        try :
            image = np.array(Image.open(img_path))
        except :
            print(img_path)
            raise
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [23]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [69]:
# train_multi_hot_for_sampler = 

# train_sampler = MultilabelBalancedRandomSampler()

train_dataset = CustomDataset(train_data['img_path'].values, np.int64(train_data['label'].values), train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val_data['img_path'].values, np.int64(val_data['label'].values), test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [70]:
next(iter(train_loader))

[tensor([[[[-0.5938, -0.4568, -0.2342,  ..., -0.8849, -0.9192, -0.2513],
           [-0.5767, -0.3712, -0.2856,  ..., -0.9534, -0.6452, -0.4226],
           [-0.5767, -0.6281, -0.3883,  ..., -0.9020, -0.7993, -0.3541],
           ...,
           [-0.6452, -0.5253, -0.4226,  ..., -0.9877, -0.5767, -0.9020],
           [-0.3883, -0.3198, -0.4054,  ..., -0.8849, -0.7822, -0.8164],
           [-0.4226, -0.5253, -0.4911,  ..., -0.7993, -0.8678, -0.6109]],
 
          [[-0.4776, -0.3375, -0.0924,  ..., -0.7752, -0.8102, -0.1275],
           [-0.4426, -0.2325, -0.1450,  ..., -0.8452, -0.5301, -0.3025],
           [-0.4426, -0.5126, -0.2675,  ..., -0.7927, -0.6877, -0.2325],
           ...,
           [-0.5301, -0.4076, -0.3025,  ..., -0.8627, -0.4426, -0.7752],
           [-0.2675, -0.1975, -0.2850,  ..., -0.7752, -0.6702, -0.7052],
           [-0.3025, -0.4076, -0.3725,  ..., -0.6877, -0.7577, -0.5126]],
 
          [[-0.2881, -0.1487,  0.0779,  ..., -0.5844, -0.6367,  0.0605],
           [-

## Model Define

In [87]:
class LightningModel(pl.LightningModule) :
    def __init__(self, num_classes = len(le.classes_)) :
        super().__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        self.criterion = nn.CrossEntropyLoss()

        self.val_preds = []
        self.val_labels = []
        self.val_loss_list = []

        # self.log_dict(CFG)

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx) :
        imgs, labels = batch

        output = self(imgs)
        loss = self.criterion(output, labels)
        self.log('train_loss', loss)

        return loss

    def configure_optimizers(self) :
        optimizer = optim.Adam(self.parameters(), lr=CFG['LEARNING_RATE'])
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
        monitor = LearningRateMonitor()
        
        return  {"optimizer": optimizer, "lr_scheduler": {'scheduler' : scheduler, 'monitor' : 'val_score'}, 'monitor' : monitor}

    def validation_step(self, batch, batch_idx) :
        imgs, labels = batch
        pred = self(imgs)
        val_loss = self.criterion(pred, labels)

        self.val_preds += pred.argmax(1).detach().cpu().numpy().tolist()
        self.val_labels += labels.detach().cpu().numpy().tolist()
        self.val_loss_list.append(val_loss.item())

    def on_validation_epoch_end(self) :
        val_loss = np.mean(self.val_loss_list)
        val_f1score = f1_score(self.val_labels, self.val_preds, average = 'weighted')
        
        self.log('val_loss', val_loss)
        self.log('val_score', val_f1score)

        self.val_loss_list.clear()
        self.val_labels.clear()
        self.val_preds.clear()

        return val_loss, val_f1score

    def predict_step(self, batch, batch_idx) :
        x = batch
        pred = self(x)
        return pred



## Train

## Run!!

In [101]:
# callbacks
ckpt_callback = ModelCheckpoint(
    monitor = 'val_score',
    dirpath = experiment_name
)

early_stop_callback = EarlyStopping(
    monitor="train_loss",
    patience=3,
    verbose=False,
    mode="min"
)


tb_logger = pl_loggers.TensorBoardLogger(save_dir = CFG['DIR'], name = CFG['EXP_NAME'])

model = LightningModel()
trainer = pl.Trainer(
    max_epochs = CFG['EPOCHS'],
    accelerator = 'auto',
    precision = 16,
    logger = tb_logger,
    callbacks = [early_stop_callback, ckpt_callback]
)

trainer.fit(model, train_dataloaders = train_loader, val_dataloaders = val_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: baseline\baseline
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | backbone   | EfficientNet     | 5.3 M 
1 | classifier | Linear           | 19.0 K
2 | criterion  | CrossEntropyLoss | 0     
------------------------------------------------
5.3 M     Trainable params
0         Non-trainable params
5.3 M     Total params
21.230    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

train\몰딩수정\29.png


## Inference

In [81]:
test = pd.read_csv('./test.csv')

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [82]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [89]:
model = LightningModel()

infer_model = model.load_from_checkpoint('experiment/baseline/epoch=0-step=76-v1.ckpt')
infer_model

LightningModel(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (sca

In [96]:
## inference

trainer = pl.Trainer()
predictions = trainer.predict(infer_model, test_loader)
final_result = []
for pred_batch in predictions :
    final_result += pred_batch.argmax(1).detach().cpu().numpy().tolist()

final_result = le.inverse_transform(final_result)


## Submission

In [98]:
submit = pd.read_csv('./sample_submission.csv')

In [99]:
submit['label'] = final_result

In [100]:
submit.to_csv('./baseline_submit.csv', index=False)